# MP4 operations

*Please make sure not to check this file into github with personal changes and costum folder paths*

To start this notebook make sure you
- installed poetry
- installed the virtual environment with `poetry install`
- selected the virtual environment as kernel for this notebook

## Imports

In [ ]:
import muvenue.services.mp4_metadata as mp4_metadata
import os
from mutagen.mp4 import MP4

## Batch Editing

In [ ]:
folder_path = 'ENTER PATH HERE'
for root, dirs, files in os.walk(folder_path):
      for file in files:
        if file.endswith('.mp4'):
            mp4_metadata.add_tag_to_mp4(os.path.join(root, file), 'ENTER TAG HERE')
            mp4_metadata.set_userrating(os.path.join(root, file), 0) # give a rating of 0 (higher = better; expect 1-10)

## Single Editing

In [ ]:
full_path = 'ENTER PATH HERE'
mp4_metadata.add_tag_to_mp4(full_path, 'ENTER TAG HERE')
mp4_metadata.set_userrating(full_path, 0) # give a rating of 0 (higher = better; expect 1-10)

## Embed thumbnails into videos

Below code creates snapshots from the videos in the folder and attaches them as thumbnail to the mp4 file.

I guess this improves the performance as the app no longer has to do this 'on the fly', but I've done it once with my files and do not run performance tests on this.

In [ ]:
import io
import moviepy.editor as mp
from mutagen.mp4 import MP4Cover
from PIL import Image

folder_path = '//mynas/Daten/Musik/Videos'
for root, dirs, files in os.walk(folder_path):
      for file in files:
        if file.endswith('.mp4'):
            if not mp4_metadata.extract_mp4_metadata(os.path.join(root, file)).get('thumbnail'):
                video = mp.VideoFileClip(os.path.join(root, file))
                snapshot = video.get_frame(3)
                image = Image.fromarray(snapshot)
                image
                img_byte_arr = io.BytesIO()
                image.save(img_byte_arr, format='PNG')
                img_byte_arr = img_byte_arr.getvalue()
                cover = MP4Cover(img_byte_arr, imageformat=MP4Cover.FORMAT_PNG)
                mp4 = MP4(os.path.join(root, file))
                mp4.tags['covr'] = [cover]
                mp4.save()
                print('Thumbnail added for ' + os.path.join(root, file))